In [1]:
import urllib.request as req
from bs4 import BeautifulSoup
from tqdm.notebook import tqdm
import pandas as pd

In [24]:
# 개별 종목

# 삼성전자, sk하이닉스, 삼성바이오로직스, NAVER, LG화학
code = ['005930', '000660', '207940', '035420', '051910']
page = [i for i in range(1, 2)]

flag = 0
inst = 0
for c in code:
    dates = []
    prices = []
    volume_p = []
    volume_i = []
    volume_f = []
    for p in page:
        url = 'https://finance.naver.com/item/frgn.nhn?code=' + c + '&page=' + str(p)
        res = req.urlopen(url)
        soup = BeautifulSoup(res, 'html.parser')
        info = soup.select('#content > div.section.inner_sub > table.type2 > tr > td > span')
        for i in range(len(info)):
            if i%9 == 0:
                dates.append(int(info[i].text.replace('.','')))
            elif i%9 == 1:
                prices.append(int(info[i].text.replace(',','')))
            elif i%9 == 5 or i%9 == 6:
                value = int(info[i].text.replace(',', ''))
                if flag:
                    volume_f.append(value)
                    value += inst
                    volume_p.append(value)
                    flag = 0
                else:
                    volume_i.append(value)
                    inst = value
                    flag = 1
    dates.reverse()
    prices.reverse()
    volume_p.reverse()
    volume_i.reverse()
    volume_f.reverse()
    pd.DataFrame({"dates":dates, "prices":prices, "volume_p":volume_p, "volume_i":volume_i, "volume_f":volume_f}).to_csv(c+'.csv', ',')

In [42]:
# KOSPI 지수, 한 페이지당 6개씩

page = [i for i in range(1,655)]
prices = []
for p in tqdm(page):
    url = 'https://finance.naver.com/sise/sise_index_day.nhn?code=KOSPI&page=' + str(p)
    res = req.urlopen(url)
    soup = BeautifulSoup(res, 'html.parser')
    info = soup.find_all("td",{"class": "number_1"})[0::4]
    for i in info:
        prices.append(float(i.text.replace(',','')))
prices.reverse()

In [41]:
# KOSPI 지수, 한 페이지당 10개씩
page = [i for i in range(1,393)]
dates = []
volume_p = []
volume_i = []
volume_f = []
for p in tqdm(page):
    url = 'https://finance.naver.com/sise/investorDealTrendDay.nhn?bizdate=20201105&sosok=&page=' + str(p)
    res = req.urlopen(url)
    soup = BeautifulSoup(res, 'html.parser')
    info = soup.select('body > table.type_1 > tr > td')[1:-2]
    del info[53:56]
    for i in range(len(info)):
        if i%11 ==0:
            dates.append(int(info[i].text.replace('.','')))
        elif i%11 == 1:
            volume_p.append(int(info[i].text.replace(',', '')))
        elif i%11 == 2:
            volume_f.append(int(info[i].text.replace(',', '')))
        elif i%11 == 3:
            volume_i.append(int(info[i].text.replace(',', '')))
dates.reverse()
volume_p.reverse()
volume_i.reverse()
volume_f.reverse()

In [63]:
pd.DataFrame({"dates":dates, "prices":prices, "volume_p":volume_p, "volume_i":volume_i, "volume_f":volume_f}).to_csv('KOSPI'+'.csv', ',')

In [62]:
dates[0]

50103

In [48]:
len(prices)

3924

In [51]:
len(dates)

3919

In [57]:
dates[-1]

50103